## Using Text Frequency - Inverse Document Frequency to Classify Recipes.
### Abstract
The algorithm TF-IDF has been used to simplify the dataset, such that common ingredients, with a high document frequency, are less priority with respect to ingredients that are more rare. The formalised equation has been shown below.

\begin{equation*}
\ w_{i,j}  = tf_{i,j} * log( \frac{N}{df_{i}})
\end{equation*}

Where w<sub>i,j</sub> is the weighted term for the word i in the document j, tf<sub>i,j</sub> is the number of occurences of that word i in the document j. N is the total number of documents, and df<sub>i</sub> is the number of occurences of the word i in all the documents. Since each ingredient will appear at most once in each recipe, the text frequency term in this formula can be replaced with 1.

In [1]:
import pandas as pd

#we need a function returning a dataframe of weighted values for each word in each document
